In [18]:
import numpy as np
import pandas as  pd

# Cutoff distance
SYSTEM="5a"
match SYSTEM:
    case "1":
        fitwnt = [32,369]
    case "3a":
        fitwnt = [19,352]
    case "4":
        fitwnt = [24,351]
    case "5a":
        fitwnt = [44,380]
    case "8a":
        fitwnt = [22,337]
    case _:
        print("Not a valid selection. Defaulting to 8a numbering.")
        fitwnt = [22,337]

refwnt = [22,337]

offset = 20

align_res100 = np.ones((fitwnt[1] - fitwnt[0] + 1, refwnt[1] - refwnt[0] + offset +1)) * 500
with open(f"output/{SYSTEM}_to_8a.out", "r") as file:
    c = 0
    for line in file:
        sp = line.split()
        align_res100[c,0:len(sp)] = sp
        c = c + 1
        
        
hashmap= []
rmsd_cutoff = 4.5
for i in range(len(align_res100)):
        fit_index = fitwnt[0]+i
        ref_index_at_min_rmsd = np.argmin(align_res100[i,1:])+align_res100[i,0]
        min_rmsd = align_res100[i,1+np.argmin(align_res100[i,1:])]
        if min_rmsd > rmsd_cutoff:
            hashmap.append([fit_index, -1, min_rmsd])
        else:
            hashmap.append([fit_index, ref_index_at_min_rmsd, min_rmsd])
        
hashmap = np.array(hashmap[:-1])

hashmap_df = pd.DataFrame(hashmap, columns=[SYSTEM,'8a','rmsd'])
hashmap_df[SYSTEM] = hashmap_df[SYSTEM].astype(int)
hashmap_df["8a"] = hashmap_df["8a"].astype(int)
hashmap_df.to_csv(f"output/{SYSTEM}_to_8a_map.csv", index=False)
hashmap_df

In [12]:
import matplotlib.pyplot as plt
plt.plot(hashmap_df[SYSTEM], hashmap_df["rmsd"])
plt.xlabel("Wnt3A Residue ID", fontsize=20)
plt.ylabel(f"Minimum RMSD ($\AA$)", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.savefig(f"{SYSTEM}_to_8a_map.png", dpi=400)